In [34]:
### Célula 1 - Funções

import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def setup_driver_with_profile(profile_path):
    """Configura o driver do Chrome com um perfil específico"""
    chrome_options = Options()
    
    # Adiciona o perfil do Chrome
    chrome_options.add_argument(f"user-data-dir={profile_path}") 
    chrome_options.add_argument("--profile-directory=Default")  # Usa o perfil padrão
    
    # Configurações adicionais
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Instala automaticamente o ChromeDriver
    service = Service(ChromeDriverManager().install())
    
    # Cria o driver com as opções configuradas
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def get_profile_image_url(driver, instagram_url):
    """Obtém a URL da imagem de perfil usando Selenium"""
    try:
        driver.get(instagram_url)
        time.sleep(3)  # Espera a página carregar
        
        # Localiza a tag de imagem do perfil
        xpath_a = '/html/body/div[1]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[1]/div/span/div/div/a/img'
        xpath_span = '/html/body/div[1]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[1]/div/span/div/div/span/img'
        
        img_element = ''

        if len(driver.find_elements(By.XPATH, xpath_a)) > 0:
            img_element = driver.find_element(By.XPATH, xpath_a)
        
        if len(driver.find_elements(By.XPATH, xpath_span)) > 0:    
            img_element = driver.find_element(By.XPATH, xpath_span)
                
        img_url = img_element.get_attribute('src')

        return img_url
    except Exception as e:
        print(f"Erro ao acessar {instagram_url}: {str(e)}")
        return None

def download_image(img_url, profile_username, output_dir='perfis'):
    """Baixa a imagem e salva no diretório especificado"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    try:
        response = requests.get(img_url, stream=True)
        if response.status_code == 200:
            file_path = os.path.join(output_dir, f"{profile_username}.jpg")
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"Imagem salva: {file_path}")
        else:
            print(f"Falha ao baixar imagem de {profile_username}")
    except Exception as e:
        print(f"Erro ao baixar imagem de {profile_username}: {str(e)}")

import csv

def load_business(caminho_arquivo):
    lista_instagram = []
    
    try:
        with open(caminho_arquivo, mode='r', encoding='utf-8') as arquivo_csv:
            leitor = csv.DictReader(arquivo_csv)

            for linha in leitor:
                instagram = linha['instagram'].replace('@', '').replace('https://www.instagram.com/', '').replace('/', '')
                lista_instagram.append({
                    'nome': linha['nome'],
                    'instagram': instagram,
                    'url': f'https://www.instagram.com/{instagram}'
                })
                
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado - {caminho_arquivo}")
    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")
    
    return lista_instagram

In [35]:
### Célula 2 - ### Lista de URLs

caminho_perfil_chrome = "C:/Users/mmmar_cz7ohi4/AppData/Local/Google/Chrome/User Data"
caminho_lista = 'instagrams.csv'
empresas = load_business(caminho_lista)

print(f"\nTotal de contas encontradas: {len(empresas)}")


Total de contas encontradas: 47


In [37]:
### Célula 3 - Inicia o driver com perfil
driver = setup_driver_with_profile(caminho_perfil_chrome)

# Se o arquivo não existia, escreve o cabeçalho
if not os.path.isfile('dados_perfis.csv'):
    with open('dados_perfis.csv', mode='w', newline='', encoding='utf-8') as arquivo_csv:
        campos = ['nome', 'instagram', 'imagem', 'url']
        escritor = csv.DictWriter(arquivo_csv, fieldnames=campos)    
        escritor.writeheader()

In [ ]:
### Célula 4 - Processar URLs

for empresa in empresas:
    print(empresa)

    # Verifica se a imagem já existe
    if os.path.exists(f'perfis/{empresa['instagram']}.jpg'):
        print(f"Imagem de {empresa['nome']} já existe. Pulando...")
        continue

    time.sleep(5)

    img_url = get_profile_image_url(driver, empresa['url'])
    
    if img_url:
        download_image(img_url, empresa['instagram'])
        empresa['imagem'] = f'{empresa['nome']}.jpg'
        
        with open('dados_perfis.csv', mode='a', newline='', encoding='utf-8') as arquivo_csv:
            escritor = csv.DictWriter(arquivo_csv, fieldnames=campos)
            escritor.writerow(empresa)
        
        print(f"Imagem de {empresa['nome']} baixada com sucesso.")        
    else:
        print(f"Não foi possível obter a imagem de {empresa['nome']}")